### 5.0. Environmental Set-up & Data Loading

In [1]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

# enter the foldername in your Drive where you have saved the unzipped
FOLDERNAME =  'ADX/'

assert FOLDERNAME is not None, 'ERROR'

%cd drive/My\ Drive
%cp -r $FOLDERNAME ../../

Mounted at /content/drive
/content/drive/My Drive


In [2]:
# import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# %matplotlib inline
# plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
# plt.rcParams['image.interpolation'] = 'nearest'
# plt.rcParams['image.cmap'] = 'gray'


In [3]:
data_ = pd.read_csv('./ADX/Data_CreditCard/creditcard.csv', encoding ='cp949', index_col = 0)
columns_ = data_.columns

In [4]:
data_.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
190368,128803.0,-2.272473,2.935226,-4.871394,2.419012,-1.513022,-0.480625,-2.126136,1.883507,-1.297262,...,0.718504,0.893850,-0.031632,0.322913,-0.058406,-0.411649,0.573803,0.176067,175.90,1
9179,13126.0,-2.880042,5.225442,-11.063330,6.689951,-5.759924,-2.244031,-11.199975,4.014722,-3.429304,...,2.002883,0.351102,0.795255,-0.778379,-1.646815,0.487539,1.427713,0.583172,1.00,1
42700,41204.0,-8.440284,6.147653,-11.683706,6.702780,-8.155839,-3.716264,-12.407313,5.626571,-6.232161,...,2.192855,-0.282597,0.008068,0.403858,-0.018788,0.522722,0.792691,0.067790,30.26,1
239501,150139.0,-6.682832,-2.714268,-5.774530,1.449792,-0.661836,-1.148650,0.849686,0.433427,-1.315646,...,0.220526,1.187013,0.335821,0.215683,0.803110,0.044033,-0.054988,0.082337,237.26,1
154286,101051.0,-1.465316,-1.093377,-0.059768,1.064785,11.095089,-5.430971,-9.378025,-0.446456,1.992110,...,1.160623,-1.259697,-15.981649,-0.883670,-3.536716,-0.592965,0.675525,0.424849,0.92,1


### 5.1. Kernel Function

Kernel function 구현

In [5]:
#################################################################################
# TODO: Kernel function Using Matrix Form  
# - Linear kernel 구현 (dot product 사용)
#################################################################################

def Kernel_(x, y, params = 0, type_ = 'default') :

    if type_ == 'rbf' :
      Kernel = np.exp(- (np.sum(x **2, axis = 1).reshape(-1,1) + np.sum(y **2, axis = 1).reshape(1,-1) - 2 * x @ y.T)* params)
    elif type_ =='default' or type_=='Linear':
      Kernel = x @ y.T
    return Kernel

    ######### type이 default일 경우 Kernel이 Linear이도록 구현 (elif 사용 + 위 rbf 구조 활용)
    

### 5.2. Linear Kernel (soft-margin) SVM using cvxopt solver

Convex Optimization Documentation:  [Link](https://cvxopt.org/userguide/coneprog.html)

---

Soft-margin SVM의 dual problem을 Convex optimization Solver를 이용하여 Alpha를 구하고, 이를 토대로 w와 b를 구한다.

**Kernel : default(dot product)**

In [6]:
from cvxopt import matrix as cvxopt_matrix
from cvxopt import solvers as cvxopt_solvers

def Convolution(pred, real) :
    pred = np.array(pred)
    y = np.array(real)
    TP = np.sum((pred == 1) & (y == 1))
    FP = np.sum((pred == 1) & (y != 1))
    FN = np.sum((pred != 1) & (y == 1))
    TN = np.sum((pred != 1) & (y != 1))
    return TP, FP, FN, TN
    
def precision_recall(X) :
  TP,FP,FN,TN = X
  return TP / (TP + FP), TP / (TP + FN)

def Minmax_(X) :
    X = np.array(X)
    return (X - X.min(axis = 0)) / (X.max(axis= 0) - X.min(axis = 0)), X.max(axis =0) , X.min(axis =0)

def Standar_(X) :
    X = np.array(X)
    return (X - X.mean(axis =0)) / X.std(axis = 0), X.mean(axis =0), X.std(axis = 0)

-------------------------------------
**Question: 상기 4개 함수의 목적과 의미를 서술하세요.**

-------------------------------------
Answer:

1. Convolution: Convolution 함수에 prediction값과 real값을 넣으면 분류 알고리즘의 성능을 평가할 수 있는 Confusion Matrix에서 쓰이는 값으로 변환한다. TP = True Positive, FP = False Positive, FN = False Negative, TN = True Negative를 의미한다.

2. precision_recall: Convolution함수에서 변환된 값을 precision_recall함수에 넣어서 순서대로 정밀도와 재현율을 반환한다.<br>
Precision = $\frac{TP}{TP + FP}$<br>
Recall = $\frac{TP}{TP + FN}$<br>

3. Minmax_: 각 array의 원소를 0과 1사이로 scaling한 값을 반환한다. 이 함수는 데이터를 정규화 할때 이용한다. 특히, 이상치의 영향을 많이 받는 데이터의 경우 사용하면 좋다.

4. Standar_: 각 array의 원소를 표준정규분포를 따르도록 정규화한 값을 반환한다. 이 함수는 Minmax와 마찬가지로 데이터를 정규화 할때 이용한다. 이 경우는 이상치의 영향이 크지 않고, 데이터의 분포가 정규성을 보일 때 이용하는 것이 좋다.
&nbsp;

In [7]:
#Initializing values and computing H matrix. Note the 1. to force to float type
C = 10  # 여기 C는 hyperparameter이므로 변형 가능함 (이번에는 10으로 고정하고 진행)
X = Standar_(np.array(data_[columns_[:-1]]))[0]
y = np.array(data_[columns_[-1]])* 1.

y = y.reshape(-1,1) 
m,n = X.shape

In [8]:
# SVM Solver 
# 1. def Kernel_ 생성, default = dot product                                    
# 2. 위 block에서 X, y, c가 정의되었을 때, H, P, q, G, h, A, b를 numpy array 형식으로 작성      
# 3. Kernel이 dot product인 경우 w와 b를 모두 출력 가능                              
# (** support vector는 0 < alpha_i< C )                                   


H = Kernel_(X, X) * 1.
H *= y@y.T

P = cvxopt_matrix(H)
q = cvxopt_matrix(-np.ones((m, 1)))
G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))
h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
A = cvxopt_matrix(y.reshape(1, -1))
b = cvxopt_matrix(np.zeros(1))

#Run solver
sol = cvxopt_solvers.qp(P, q, G, h, A, b)
alphas = np.array(sol['x'])

     pcost       dcost       gap    pres   dres
 0: -5.8129e+02 -1.0298e+05  4e+05  1e+00  7e-13
 1: -4.2201e+02 -5.1005e+04  1e+05  3e-01  6e-13
 2: -2.6726e+02 -2.0455e+04  4e+04  9e-02  4e-13
 3: -1.8589e+02 -1.0262e+04  2e+04  4e-02  2e-13
 4: -9.6012e+01 -5.7119e+03  8e+03  1e-02  2e-13
 5: -8.7623e+01 -2.4313e+03  3e+03  4e-03  2e-13
 6: -1.3650e+02 -1.2150e+03  1e+03  1e-03  2e-13
 7: -1.7550e+02 -6.5484e+02  5e+02  3e-04  2e-13
 8: -1.9086e+02 -5.3416e+02  3e+02  1e-04  2e-13
 9: -2.0543e+02 -4.3445e+02  2e+02  5e-05  2e-13
10: -2.1462e+02 -4.0309e+02  2e+02  3e-05  2e-13
11: -2.2073e+02 -3.7532e+02  2e+02  2e-05  2e-13
12: -2.3983e+02 -3.3847e+02  1e+02  5e-06  3e-13
13: -2.4876e+02 -3.1464e+02  7e+01  3e-06  2e-13
14: -2.4776e+02 -3.1241e+02  6e+01  2e-06  2e-13
15: -2.5088e+02 -3.0203e+02  5e+01  2e-06  2e-13
16: -2.5455e+02 -2.9130e+02  4e+01  9e-07  2e-13
17: -2.5610e+02 -2.8591e+02  3e+01  6e-07  2e-13
18: -2.6030e+02 -2.7701e+02  2e+01  1e-07  2e-13
19: -2.6374e+02 -2.71

-------------------------------------
**Question: P, q, G, h, A, b가 각각 의미하는 바를 cvxopt_solver와 Soft-margin SVM의 Dual 식과 연관하여 서술하세요.**

(Hint:  각각의 정의는 Soft-margin SVM의 Dual 식과 미세한 차이가 존재함. 그 이유에 대해서도 반드시 고민하여 함께 서술할 것.)

-------------------------------------

Answer:

P는 Soft-margin SVM의 Dual식에서 H를 의미한다.

x는 Dual식에서 alpha를 의미하고
q의 경우는 m * 1 형태의 -1로, x(Dual식에서의 alpha)와 곱해져 alpha_i의 summation에 -1을 곱한 것을 의미한다.

Dual식에서의 부호와 반대인 이유는 cvxopt에서 quadratic programming은 minimize문제이고, Dual식에서는 maximize 문제이기에 -1을 곱해 부호를 바꿔준 것이다. 

G는 m * m 형태의 단위행렬에 -1을 곱한 것과 m * m 형태의 단위행렬을 위아래로 붙인 형태로, 2m * m 행렬이며 h는 m개의 0에 m개의 C가 가로로 붙은 2m * 1 행렬이다.
cvxopt에서 이는 Gx<=h 제약을 나타내는데, 이는 Dual식에서의 0<=alpha_i<=C 제약과 같은 의미이다.

A는 1 * m 형태인 y 행렬, b는 0인 스칼라값이며, Dual식에서의 Summation(alpha_i * y_i) = 0 제약을 의미한다. 

In [9]:
#Results

w = ((y * alphas).T @ X).reshape(-1,1)
S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
b = y[S] - np.sum(Kernel_(X, X[S], type_ = 'default')* y * alphas , axis = 0).reshape(-1,1)

print('Alphas = ',alphas[(alphas > 1e-4) & (alphas < C-1e-4)])
print('')
print('w = ', w.flatten())
print('')
print('b = ', np.mean(b))
print('')
print("support vector : ", np.array(range(m))[S])

Alphas =  [0.93391227 0.73167443 9.07800516 9.43345917 0.20215315 4.75997645
 0.04898054 0.9437631  1.86790295 0.47439464 3.11719337 4.15402778
 0.69513092 6.68553408 2.71619507 0.50311204 3.00745515 0.39546727
 7.15310869 9.09066684 7.95663166 3.62974161 5.19419453 1.73525492
 7.67969291 1.70451724 0.97884382 3.32793671 1.40845859 1.15099831]

w =  [-0.16485571  1.51871316 -0.0486063   0.70754104  1.13478361  0.91858635
 -0.40166639 -1.39914304 -0.99450461 -1.44138494 -2.46660068  0.33934072
 -1.34516895 -0.15421702 -1.75442775 -0.11663225 -0.85224686 -1.15180705
 -0.59394569  0.02060396 -1.26201162  0.11997614  0.12094323  0.17223573
  0.18050741 -0.24004218 -0.3830853  -1.02168134  0.42591414  1.06549356]

b =  -1.187055243842712

support vector :  [  4  23  48  65  96 120 143 146 283 295 310 313 318 339 352 445 462 470
 516 538 565 610 659 671 743 748 757 826 836 870]


-------------------------------------
**Question: Alphas, w, b, support vector가 의미하는 바를 각각 수리적 관점에서 서술하세요.**

-------------------------------------

Answer:

이 문제에서 우리가 구하고자 하는 것은 데이터를 분류해줄 수 있는 hyperplane이다. hyperplane은 y = w.T * x + b 의 방정식으로 나타낼 수 있는데 이 방정식을 찾기 위해 support vector를 사용할 수 있다. support vector란 hyperplane 위에 있는 점으로 이 점의 정보를 알고 있다면 hyperplane의 방정식을 구할 수 있다. 이 때, support vector를 구하기 위해 Alpha값을 이용하면 된다. Complementary slackness에 의해 주어진 데이터가 hyperplane 위에 있다면 Alpha 값이 존재하고, hyperplane 위에 있지 않다면 Alpha 값은 0이 되기 때문이다.

우리는 KKT조건에 의해 w = SUM(alpha * y * x) 임을 알고 있기 때문에, hyperplane위에 있는 점들(Alpha값이 존재하는 support vector)을 y=w.T * x+b 방정식에 대입하여 b 값을 얻어낼 수 있다. 최종적으로 w와 b를 구했으므로 hyperplane의 방정식을 찾아낸 것이다.

본 문제의 경우 hyperplane 위에 있는 support vector가 총 30개이고 따라서 hyperplane은 30개 점들의 선형결합으로 표현된다.

In [10]:
# Scikit Learn 라이브러리 내 SVM 적용

from sklearn.svm import SVC
clf = SVC(C = 10, kernel = 'linear', tol = 1e-5)
clf.fit(X, y.ravel()) 

print('Coefficients of the support vector in the decision function = ')
print(np.abs(clf.dual_coef_))
print('')
print('w = ',clf.coef_)
print('')
print('b = ',clf.intercept_)
print('')
print('support vector = ', clf.support_)

Coefficients of the support vector in the decision function = 
[[ 4.76006589  0.04894641  0.94363037  1.86779681  0.47456799  3.11692477
   4.15436147  0.69502833 10.          6.68540801  2.71612202 10.
   0.50261854  3.00725304  0.39548292  7.15298827  9.0907824   7.95709443
  10.          3.63028522  5.19409322  1.73513001 10.          7.67990817
   1.70454686  0.97825134  3.32861164  1.40849383 10.          1.15113552
  10.          0.93389653 10.         10.          0.7315151  10.
  10.         10.         10.         10.         10.         10.
   9.07800822 10.         10.          9.4339484   0.20215923 10.        ]]

w =  [[-0.16486385  1.5187921  -0.04858539  0.70753091  1.13475296  0.91864415
  -0.40165293 -1.39919689 -0.99447836 -1.44142932 -2.46657873  0.33933867
  -1.34522562 -0.15421595 -1.75448367 -0.1166373  -0.85228423 -1.15186252
  -0.59392978  0.02060769 -1.26206482  0.11998439  0.12095367  0.17225002
   0.18051531 -0.2400559  -0.38308163 -1.02168915  0.42592805  1.

In [11]:
# Classification based on Linear Kernel SVM 
# - alpha, b를 이용하여 예측값 도출 (np.sign, Kernel 활용)                     #
pred_sol = np.sign(np.sum(Kernel_(X, X  ,type_ = 'default')* y * alphas , axis = 0).reshape(-1,1) + b[0])

# Scikit Learn 결과 와의 비교
np.sum(clf.predict(X) - pred_sol.flatten())

0.0

In [12]:
# Classification Performance based on Linear Kernel SVM 
print(precision_recall(Convolution(clf.predict(X),y.flatten())))  # scikit-learn 정밀도 재현율, 차원을 맞추기 위해 flatten()
print(precision_recall(Convolution(pred_sol,y))) # 구현한 모델의 정밀도 재현율

(0.9891304347826086, 0.91)
(0.9891304347826086, 0.91)


In [13]:
print('실제 1의 개수:',len(y[y==1]))
print('실제 -1의 개수',len(y[y==-1]))
print('예측된 1의 개수',len(pred_sol[pred_sol==1]))
print('예측된 -1의 개수',len(pred_sol[pred_sol==-1]))

실제 1의 개수: 100
실제 -1의 개수 900
예측된 1의 개수 92
예측된 -1의 개수 908


-------------------------------------
**Question: 상기 classification performance의 의미를 서술하세요.**

-------------------------------------

Answer:

결과값의 Precision은 True로 분류된 것 중에서 실제 True의 비율을 의미하고 0.989의 값이 나왔다. 92개의 True라고 분류된 것 중에서 실제로 True인 개수는 92에 precision값을 곱한것으로 표현되므로 91개는 제대로 1로 분류되었고, 1개는 잘 못 분류되었다고 볼 수 있다. Recall은 실제 True값 중에서 True라고 예측한 비율이고 0.91의 값이 나왔다. 이는 실제 1의 개수 100개중에 91개는 1로 제대로 예측하였고 9개는 잘 못 예측했다고 볼 수 있다.



### 5.3. Gaussian (RBF) Kernel SVM using cvxopt solver

Kernel로 미리 정의해 둔 RBF 사용하여 직접 구현할 것

In [14]:
#################################################################################
# TODO: SVM Solver Kernel = 'rbf'                                 
# 1. Linear Kernel과 동일하게 H matrix 계산을 위한 기초값 Initialization
# 2. def Kernel_ 작성, default = rbf product                      
# 3. H, P, q, G, h, A, b를 numpy array 형식으로 작성 (Gamma = 2로 설정)
# 4. Alphas, S, b 결과 출력 (Kernel이 rbf인 경우 b를 출력 (w는 출력 불가)
#################################################################################

# module implementation

class new_SVC:
  def __init__(self, gamma = 0, type_ = 'default'):
    self.gamma = gamma
    self.type_ = type_
    self.kernel = None
    self.alphas = None
    self.S = None
    self.y = None
    self.w = None
    self.b = None

  #2
  def Kernel_(self, x, y, gamma, type_ = 'default') :
    if type_ == 'rbf' :
      Kernel = np.exp(-(np.sum(x**2, axis = 1).reshape(-1, 1) + np.sum(y**2, axis = 1).reshape(1, -1) - 2 * x @ y.T)* gamma)
    elif type_ == 'default' or type_ == 'linear':
      Kernel = x @ y.T
    return Kernel


  def getArgs(self, X, y):
    Kernel = self.Kernel_(X, X, self.gamma, self.type_)
    self.kernel = Kernel

    #1
    H = Kernel * 1.
    H *= y @ y.T

    #3
    P = cvxopt_matrix(H)
    q = cvxopt_matrix(-np.ones((m, 1)))
    G = cvxopt_matrix(np.vstack((-np.eye(m),np.eye(m))))
    h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * C)))
    A = cvxopt_matrix(y.reshape(1, -1))
    b = cvxopt_matrix(np.zeros(1))

    return P, q, G, h, A, b

  
  def solver(self, X, y):
    P, q, G, h, A, b = self.getArgs(X, y)
    sol = cvxopt_solvers.qp(P, q, G, h, A, b)

    alphas = np.array(sol['x'])
    self.alphas = alphas

    if self.type_ == 'default':
      w = ((y * alphas).T @ X).reshape(-1, 1)
    else:
      w = None # rbf는 w 출력 불가
    self.y = y
    S = ((alphas > 1e-4) & (alphas < C-1e-4)).flatten()
    if self.type_ == 'rbf':
      b = y[S] - np.sum(self.Kernel_(X, X[S], self.gamma, self.type_) * y * alphas, axis = 0).reshape(-1, 1) # rbf의 경우 b를 출력
    else:
      b = None

    self.w = w
    self.S = S
    self.b = b

    return alphas, S, w, b

  def predict(self, X):
    pred_sol = np.sign(np.sum(self.Kernel_(X, X, self.gamma, self.type_) * self.y * self.alphas, axis = 0).reshape(-1,1) + np.mean(self.b))
    return pred_sol


In [15]:
rbf_svm = new_SVC(2, 'rbf')
alphas, S, w, b = rbf_svm.solver(X, y)

#4
print('alphas(rbf) = ')
print(alphas)
print()

print('S(rbf) = ')
print(S)
print()

print('b = ')
print(b)
print()
  

     pcost       dcost       gap    pres   dres
 0:  1.1233e+03 -2.1902e+04  2e+04  1e-12  5e-15
 1:  4.0674e+02 -3.1858e+03  4e+03  2e-13  4e-15
 2: -1.3134e+02 -4.7529e+02  3e+02  4e-13  1e-15
 3: -1.7599e+02 -2.0145e+02  3e+01  1e-13  4e-16
 4: -1.7673e+02 -1.7721e+02  5e-01  8e-14  9e-17
 5: -1.7674e+02 -1.7678e+02  4e-02  2e-13  6e-17
 6: -1.7674e+02 -1.7674e+02  2e-03  2e-13  7e-17
 7: -1.7674e+02 -1.7674e+02  4e-05  4e-14  6e-17
Optimal solution found.
alphas(rbf) = 
[[1.79257312e+00]
 [1.79257312e+00]
 [1.79257311e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79256632e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79353230e+00]
 [1.17711994e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.74407303e+00]
 [1.79256950e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.79257312e+00]
 [1.74407303e+00]
 [1.79257276e+00]
 [1.79257312e+00]

In [16]:
#################################################################################
# TODO: Scikit Learn으로 Gaussian Kernel SVM 구현
# - 상기 구현과 같은 parameter 넣을 것 (C와 Gamma)
# - 그 외 input은 Linear Kernel SVM과 동일하게 적용
#################################################################################

# Scikit Learn 라이브러리 내 SVM 적용

clf_rbf = SVC(C = 10, kernel = 'rbf', tol = 1e-5, gamma = 2)
clf_rbf.fit(X, y.ravel())
clf_rbf_pred = clf_rbf.predict(X) #scikit-learn 모델의 predict값

print('support vector(sklearn) = ', clf_rbf.support_)
print('')
print('b(sklearn) = ', clf_rbf.intercept_)
print('')
print('prediction(sklearn) = ', clf_rbf_pred)

support vector(sklearn) =  [100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117
 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153
 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171
 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189
 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207
 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225
 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243
 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261
 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280
 281 282 283 284 285 286 287 288 289 290 291 292 293 294 295 296 297 298
 299 300 301 302 303 304 305 306 307 308 309 310 311 312 313 314 315 316
 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334
 335 336 337 338 339 340

In [17]:
###############################################################################################
# TODO: Classification 시행 및 Scikit learn과의 비교 시행 (차이가 존재하지 않는 것을 확인할 것)
###############################################################################################

# 예측값 도출                   #
pred_sol_ = rbf_svm.predict(X) # 구현 모델

# Scikit Learn 결과 와의 비교
np.sum(clf_rbf_pred - pred_sol_.flatten())

0.0

In [18]:
#################################################################################
# TODO: 직접 구현 한 Gaussian Kernel SVM의 Classification Performance 계산
#################################################################################

# Classification Performance based on Gaussian Kernel SVM
print(precision_recall(Convolution(clf_rbf_pred, y.flatten()))) # scikit-learn 정밀도 재현율, 차원을 맞추기 위해 flatten()
print(precision_recall(Convolution(pred_sol_, y))) # 구현된 모델의 정밀도 재현율

(1.0, 1.0)
(1.0, 1.0)


In [19]:
# 구현한 모델들의 rbf, linear kernel에 대한 정밀도와 재현율

print('(Precision, Recall) of rbf kernel:')
print(precision_recall(Convolution(pred_sol_, y)))
print()
print('(Precision, Recall) of Linear kernel')
print(precision_recall(Convolution(pred_sol, y)))

(Precision, Recall) of rbf kernel:
(1.0, 1.0)

(Precision, Recall) of Linear kernel
(0.9891304347826086, 0.91)


-------------------------------------
**Question: Linear와 Gaussian Kernel SVM의 Performance 차이를 논하세요.**

-------------------------------------

Answer: Linear kernel SVM의 경우에는 결과값의 Precision(정밀도)이 0.989, Recall(재현율)은 0.91의 값이 나왔다. 반면에 Gaussian kernel의 경우에는 결과값이 모두 1.0이라는 float 값으로 나오게 되었다. 즉, 정밀도와 재현율 값이 Linear kernel에 비해 Gaussian kernel이 높게 나왔기에, 이 정량적인 수치 비교 측면에서 Gaussian kernel의 성능이 더 우수하다 할 수 있다. 